In [1]:
import torch, numpy as np, torch_geometric, random

In [2]:
device = torch.device("cuda")

In [3]:
class AltGraphConv(torch.nn.Module):
    def __init__(self):
        super(AltGraphConv, self).__init__()
        self.gcn1 = torch_geometric.nn.GATConv(32, 16, edge_dim=2)
        self.gcn2 = torch_geometric.nn.GATConv(16, 1, edge_dim=2)
    def forward(self, mat, inps):
        h = self.gcn1(mat, inps).tanh()
        return self.gcn2(h, inps).tanh()

In [4]:
class AMDGCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = torch_geometric.nn.GATConv(16, 32, edge_dim=2)
        self.gcn2 = torch_geometric.nn.GATConv(32, 32, edge_dim=2)
        self.gcn3 = torch_geometric.nn.GATConv(32, 32, edge_dim=2)
        self.gcn4 = AltGraphConv()
        self.global_pool = torch_geometric.nn.aggr.SortAggregation(k=30)
        self.conv1 = torch.nn.Conv1d(1, 16, 97, 97)
        self.conv2 = torch.nn.Conv1d(16, 32, 5, 1)
        self.maxpool = torch.nn.MaxPool1d(2, 2)
        self.linear1 = torch.nn.Linear(352, 128)
        self.linear2 = torch.nn.Linear(128, 1)
        self.dropout = torch.nn.Dropout(0.5)
    def forward(self, mat, edge_index):
        h1 = self.gcn1(mat, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)
        h = self.global_pool(h)
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()
        return h

In [5]:
model = AMDGCNN()
model.train(True)
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr = 0.0005623324269475477)
criterion = torch.nn.BCELoss()

In [6]:
def train_one_epoch(k):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    pos_batch_dat = np.load(f'pos_dat/pos_batch_{k}.npz', allow_pickle=True)
    neg_batch_dat = np.load(f'neg_dat/neg_batch_{k}.npz', allow_pickle=True)
    pos_edge_dat = np.load(f'pos_dat/pos_edge_attr_{k}.npz', allow_pickle=True)
    neg_edge_dat = np.load(f'neg_dat/neg_edge_attr_{k}.npz', allow_pickle=True)
    pos_table = pos_batch_dat.f.arr_0
    neg_table = neg_batch_dat.f.arr_0
    pos_edges = pos_edge_dat.f.arr_0
    neg_edges = neg_edge_dat.f.arr_0
    lab_t = [j for j in range(len(pos_table)+len(neg_table)-1)]
    random.shuffle(lab_t)
    print(f"Batch {k} in progress")
    ind = 0
    for i in lab_t:
        label = None
        g = None
        x = None
        e = None
        if i < len(pos_table):
            label = torch.tensor([1.], dtype=torch.float32).to(device)
            g = pos_table[i][2]
            x = pos_table[i][3]
            e = pos_edges[i]
        else:
            label = torch.tensor([0.], dtype=torch.float32).to(device)
            g = neg_table[i - len(pos_table)][2]
            x = neg_table[i - len(pos_table)][3]
            e = neg_edges[i - len(pos_table)]
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        inp = torch.sparse_coo_tensor(torch.tensor(g, dtype=torch.float32).to_sparse_coo().indices(), torch.tensor(e, dtype = torch.float32)).coalesce().to(device)
        outputs = model( torch.tensor(x).to(torch.float32).to(device), inp)
        #inp = torch.sparse_coo_tensor(torch.tensor(g, dtype=torch.float32).abs().to_sparse_coo().indices(), torch.tensor(e, dtype=torch.float32)).to(device)
        #outputs = model(torch.tensor(x).to(torch.float32).to(device), inp)


        # Compute the loss and its gradients
        loss = criterion(outputs.view(1), label)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        ind +=1
        running_loss += loss.item()
        if ind == 197:
            last_loss = running_loss # loss per batch
            print(f'Output: {outputs}, label: {label}')
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
    return last_loss

In [7]:
for epoch in range(8):
    print(f"Epoch {epoch} in progress")
    for i in range(1, 31):
        train_one_epoch(i)

Epoch 0 in progress
Batch 1 in progress
Output: tensor([[0.5603]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 130 loss: 133.43557077646255
Batch 2 in progress
Output: tensor([[0.6139]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 118 loss: 105.06662209425122
Batch 3 in progress
Output: tensor([[0.1806]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 166 loss: 42.75910749965391
Batch 4 in progress
Output: tensor([[0.1130]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 115 loss: 14.923920023666469
Batch 5 in progress
Output: tensor([[0.7191]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([1.], device='cuda:0')
  batch 85 loss: 55.290898809327054
Batch 6 in progress
Output: tensor([[0.9998]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([1.], device='cuda:0')
  batch 77 loss: 

In [8]:
#test_inds = [8,9,10,11,12,13,14,15,16]
actuals = []
expecteds = []
probs = []

In [9]:
torch.no_grad()
model.train(False)
for k in range(31,40):
    pos_batch_dat = np.load(f'pos_dat/pos_batch_{k}.npz', allow_pickle=True)
    neg_batch_dat = np.load(f'neg_dat/neg_batch_{k}.npz', allow_pickle=True)
    pos_edge_dat = np.load(f'pos_dat/pos_edge_attr_{k}.npz', allow_pickle=True)
    neg_edge_dat = np.load(f'neg_dat/neg_edge_attr_{k}.npz', allow_pickle=True)
    pos_table = pos_batch_dat.f.arr_0
    neg_table = neg_batch_dat.f.arr_0
    pos_edges = pos_edge_dat.f.arr_0
    neg_edges = neg_edge_dat.f.arr_0
    lab_t = [j for j in range(len(pos_table)+len(neg_table)-1)]
    random.shuffle(lab_t)
    print(f"Batch {k} in progress")
    ind = 0
    outs = []
    temp_l = []
    for i in lab_t:
        label = None
        g = None
        x = None
        e = None
        if i < len(pos_table):
            temp_l.append(1)
            g = pos_table[i][2]
            x = pos_table[i][3]
            e = pos_edges[i]
        else:
            temp_l.append(0)
            g = neg_table[i - len(pos_table)][2]
            x = neg_table[i - len(pos_table)][3]
            e = neg_edges[i - len(pos_table)]

        # Make predictions for this batch
        inp = torch.sparse_coo_tensor(torch.tensor(g, dtype=torch.float32).to_sparse_coo().indices(), torch.tensor(e, dtype = torch.float32)).coalesce().to(device)
        outputs = model( torch.tensor(x).to(torch.float32).to(device), inp)
        print(outputs)
        o = 0
        if outputs.view(1)[0] > 0.5:
            o = 1
        outs.append(o)
        probs.append(float(outputs.view(1)[0]))
        torch.cuda.empty_cache()
    expecteds.extend(temp_l)
    actuals.extend(outs)
    torch.cuda.empty_cache()



Batch 31 in progress
tensor([[0.9993]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9804]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9809]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[3.8917e-05]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9993]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[7.4756e-06]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9928]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[6.4879e-06]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9812]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[2.6570e-08]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[1.7041e-06]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9909]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[5.6201e-06]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9791]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[1.4785e-05]], device='cuda:0', grad_fn=<SigmoidBac

In [10]:
torch.save(model.state_dict(), './ADGCNN_m')

In [ ]:
model.load_state_dict(torch.load('./model'))

<All keys matched successfully>

In [ ]:
model.train(True)

SEAL(
  (gcn1): GCNConv(16, 32)
  (gcn2): GCNConv(32, 32)
  (gcn3): GCNConv(32, 32)
  (gcn4): GCNConv(32, 1)
  (global_pool): SortAggregation(k=30)
  (conv1): Conv1d(1, 16, kernel_size=(97,), stride=(97,))
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=352, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
torch.enable_grad()

In [11]:
sum = 0.
for l in range(len(expecteds)):
    if expecteds[l] == actuals[l]:
        sum += 1
sum/len(actuals)

0.9278059785673999

In [12]:
from sklearn.metrics import roc_curve, auc

In [13]:
fpr, tpr, thr = roc_curve(np.array(expecteds).astype(np.int64), np.array(probs).astype(np.float64), pos_label=1)

In [14]:
auc(fpr, tpr)

0.9949825796386643